# Library

In [4]:
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

import os
import random
import numpy as np
from gensim.models import word2vec #  word2vec library  
import nltk
import matplotlib as mpl
import matplotlib.pyplot as plt
import sentencepiece as spm # For sentencepiece model

# Load Data

In [2]:
data_path = '../../../기타/data/klue/'

train = pd.read_csv(data_path + 'train_data.csv')
test = pd.read_csv(data_path + 'test_data.csv')

In [3]:
train.head()

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral


## Set Data

In [26]:
with open('premise_train.txt', 'w') as f : 
    for row in train.iterrows() : 
        txt = row[1]['premise'] +'\n'
        f.write(txt)
f.close()

with open('hypothesis_train.txt', 'w') as f : 
    for row in train.iterrows() : 
        txt = row[1]['hypothesis'].replace('\xa0',' ') +'\n'
        f.write(txt)
f.close()

# Sentence Piece & Word 2 Vec

## Set Parameters

In [27]:
num_features = 300 # Embedding Vector Size
negative = 10 # words for negative sampling
min_word_count = 6 # minimum words in one sentence
window = 7 # context window size
downsampling = 0.75 # Lower frequency for high-frequency words 
epoch = 20

In [30]:
%%time

spm.SentencePieceTrainer.train(input="./premise_train.txt", model_prefix='pre', vocab_size=3000, user_defined_symbols=['[CLS]', '[SEP]'])
pre_sp = spm.SentencePieceProcessor(model_file='./pre.model')

Wall time: 595 ms


In [32]:

spm.SentencePieceTrainer.train(input="./hypothesis_train.txt", model_prefix='hypo', vocab_size=1500, user_defined_symbols=['[CLS]', '[SEP]'])
hypo_sp = spm.SentencePieceProcessor(model_file='./hypo.model')

In [61]:
sp_premises = []
premises = []
for line in train.iterrows() : 
    line = line[1]['premise']
    premises.append(line.split(' '))
    sp_premises.append(pre_sp.encode(line, out_type=str))

sp_hypos = []
hypos = []
for line in train.iterrows() : 
    line = line[1]['hypothesis']
    hypos.append(line)
    sp_hypos.append(hypo_sp.encode(line, out_type=str))

In [62]:
%%time

# skip-gram model training with naive splitted data
premises_normal = word2vec.Word2Vec(premises,
                                sg = 1, # CBOW = 0, skip-gram = 1
                                negative=negative,
                                size=num_features, 
                                min_count=min_word_count,
                                window=window,
                                sample=downsampling,
                                iter=epoch)


Wall time: 18.4 s


In [ ]:
%%time
# skip-gram model training with sentencepiece data
premises_SP = word2vec.Word2Vec(sp_premises,
                                  sg = 1, # CBOW = 0, skip-gram = 1
                                  negative=negative,
                                  size=num_features, 
                                  min_count=min_word_count,
                                  window=window,
                                  sample=downsampling,
                                  iter=epoch)

In [64]:
premises_normal.wv.most_similar('형사')

[('고소', 0.8074188232421875),
 ('찾게', 0.7560069561004639),
 ('4명을', 0.7133421897888184),
 ('사인을', 0.7122508883476257),
 ('수십억', 0.7120131850242615),
 ('12일,', 0.6983972787857056),
 ('좋게', 0.6958245038986206),
 ('강력', 0.6878354549407959),
 ('사망한', 0.6872656941413879),
 ('농민의', 0.6797583103179932)]

In [63]:
premises_SP.wv.most_similar('형사')

[('고소', 0.7749349474906921),
 ('부는', 0.6798735857009888),
 ('수십억', 0.6695069670677185),
 ('벌인', 0.6201853156089783),
 ('수천만', 0.6008221507072449),
 ('병사', 0.588842511177063),
 ('명에게', 0.588783323764801),
 ('살인죄를', 0.5843274593353271),
 ('예고', 0.5793607234954834),
 ('측이', 0.5697324275970459)]

In [65]:
train.head()

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral


In [68]:
premises_normal.wv.most_similar('남자')

[('테니스', 0.7148362994194031),
 ('남편과', 0.6501109600067139),
 ('포켓몬', 0.6459095478057861),
 ('리그', 0.6060731410980225),
 ('없이도', 0.6030577421188354),
 ('실수는', 0.5894483923912048),
 ('21살의', 0.5875996351242065),
 ('오갈', 0.5752477049827576),
 ('비센테는', 0.5663378834724426),
 ('위협을', 0.5608822107315063)]

In [69]:
premises_SP.wv.most_similar('남자')

[('테니스', 0.7378988265991211),
 ('실수는', 0.7027692794799805),
 ('비센테는', 0.6894849538803101),
 ('중국인', 0.6736307740211487),
 ('위협을', 0.6219642162322998),
 ('최대의', 0.591732919216156),
 ('이중으로', 0.5772334337234497),
 ('송은', 0.5688022375106812),
 ('이란', 0.5652413368225098),
 ('로맨스가', 0.5635850429534912)]